##### In this homework, we will learn more about `function calling`, and we will also explore `MCP - model-context protocol`.



### Preparation

First, we'll define a function that we will use when building our agent.

It will generate fake weather data:

In [5]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description
We want to use it as a tool for our agent, so we need to describe it

How should the description for this function look like? Fill in missing parts


``` python
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}
```

# What’s the goal?

we’re preparing to wrap your Python function `get_weather(city: str)` into a machine-readable schema — something an AI agent (like a chatbot or LLM) can understand and call.

This is part of a function-calling interface where the agent needs:

- the name of the function


- a description of what it does


- what parameters it accepts


- the types and descriptions of those parameters


That way, when a user asks “What’s the weather in Paris?”, the agent knows:

There's a tool called get_weather

It takes a parameter named "city" which is a string

It should call get_weather(city="paris") to get the result


```python

get_weather_tool = {
    "type": "function",
    "name": "get_weather",  # TODO1
    "description": "Get the current weather for a given city.",  # TODO2
    "parameters": {
        "type": "object",
        "properties": {
            "city": {  # TODO3
                "type": "string",
                "description": "The name of the city to get the weather for."  # TODO4
            }
        },
        "required": ["city"],  # TODO5
        "additionalProperties": False
    }
}

```

# `Q1-Answer -> city`

### Q2. Adding another tool

Let's add another tool - a function that can add weather data to our database:



In [6]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

# `Q2-Answer here`

In [7]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Adds or updates the weather data for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to store weather data for."
            },
            "temp": {
                "type": "number",
                "description": "The temperature value to be stored for the city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}


### MCP

MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

A tool can export a list of functions it has
When we include the tool to our Agent, we just need to include the link to the MCP server


> MCP lets large language models (LLMs) like GPT connect to external tools (e.g. Qdrant) over the network.

> Instead of manually defining each function, you just provide a URL to the MCP server, and the model automatically:

-  Discovers what functions the tool supports

- Learns how to call them

- Uses them dynamically

##### example

`agent.include_tool("https://your-tool.com/mcp")`

### Q3. Install FastMCP

Let's install a library for MCP 

```python 
pip install fastmcp
```

In [8]:
%pip install fastmcp

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=19270) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


# `Q3-Answer -> 2.10.5`

### Q4. Simple MCP Server

A simple MCP server from the documentation looks like that:



# weather_server.py
``` python 
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()
```  

### For our case

``` python
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

```

### I'm using `weather_server.py` and `mcp_client.py` from this point forward

# `Q4: Answer -> http`

In [9]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


# `Q5-Answer`

```json

{
  "jsonrpc": "2.0",
  "id": 3,
  "result": {
    "result": 20.0
  }
}
```

# `Q6-Answer`

### `mcp_client.py` will show this output.

```json

Available tools: [Tool(name='get_weather_tool', title=None, description=None, inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather_tool', title=None, description=None, inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None)]
Weather in Berlin: CallToolResult(content=[TextContent(type='text', text='20.0', annotations=None, meta=None)], structured_content={'result': 20.0}, data=20.0, is_error=False)
Set weather response: CallToolResult(content=[TextContent(type='text', text='OK', annotations=None, meta=None)], structured_content={'result': 'OK'}, data='OK', is_error=False)
Weather in Paris: CallToolResult(content=[TextContent(type='text', text='25.5', annotations=None, meta=None)], structured_content={'result': 25.5}, data=25.5, is_error=False)

```